# Advanced House Prices Regression

Objective: to predict the House Prices based on relevant features.

Supervised Learning: Regression

In [ ]:
!pip install --upgrade scikit-learn

In [ ]:
# Import Library

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, root_mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder

from sklearn.pipeline import Pipeline
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.metrics import mutual_info_score 
from sklearn.impute import SimpleImputer
import scipy.stats as stats
from sklearn.model_selection import cross_val_score

import optuna
import optuna.visualization as vis

import warnings
warnings.filterwarnings('ignore')

# Set the maximum number of columns and rows to display to a large number
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# graphic settings

sns.set_theme(rc={'figure.figsize':(20.7, 18.27)})
sns.set_style("whitegrid")
sns.color_palette("dark")
plt.style.use("fivethirtyeight")

In [ ]:
# Import Dataset

df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")

### Dataset Information Overview

In [ ]:
df.head()

In [ ]:
df.info()

Some columns have missing values. We will deal with this later in data preprocessing. 

In [ ]:
df.describe()

In [ ]:
df.describe(include='object')

Some features such as Street and Utilities dominated the class, we will check the variance in feature selection step. 

### Train Test Data Split

In [ ]:
# drop Id column because it is not used

df.drop('Id', axis = 1, inplace=True)

df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

## Exploratory Data Analysis

In [ ]:
# separate variables into 3 categories based on data types
# numerical features with unique values lower than 20 will be categorized as discrete variables. 

categorical = [col for col in df_train.columns if df[col].dtype not in ['float64','int64']]
discrete = [col for col in df_train.columns if df[col].nunique() < 20 and col not in categorical]
numerical = [col for col in df_train.columns if col not in categorical + discrete + ['SalePrice']]

target = 'SalePrice' # for EDA purpose

We will perform Exploratory Data Analysis for each features data types: categorical, discrete, and numerical. 

### Categorical Variables

In [ ]:
df_train_categorical = df_train[categorical]

In [ ]:
df_train[categorical].describe()

#### Data Distribution for Categorical Variables

In [ ]:
# distribution check in categorical variables

for i in range(0, len(df_train_categorical.columns)):
    plt.subplot(8, 6, i+1)
    sns.countplot(x=df_train_categorical[categorical[i]])
    plt.title(categorical[i])
    x=plt.xticks(rotation=90)
    plt.suptitle('Data Distribution in Categorical Variables', fontsize = 20)
    plt.tight_layout()

Some of features have a dominant unique value, such as Street, CentralAir, and Electrical. Neighborhood have the highest number of unique values. 

#### Relation between Categorical Variables and Target

In [ ]:
for i in range(0, len(df_train_categorical.columns)):
    plt.subplot(8, 6, i+1)
    sns.boxplot(x=df_train_categorical[categorical[i]], y = target, data = df_train)
    plt.title(categorical[i])
    x=plt.xticks(rotation=90)
    plt.suptitle('Relation between Categorical Variables and Target', fontsize = 20)
    plt.tight_layout()

There are some features such as ExterQual, BsmtQual, and MiscFeature have significant difference of Sale Price between its category. 

### Discrete Variables

In [ ]:
df_train_discrete = df_train[discrete]

#### Data Distribution for Discrete Features

In [ ]:
for i in range(0, len(df_train_discrete.columns)):
    plt.subplot(4, 4, i+1)
    sns.countplot(x=df_train_discrete[discrete[i]])
    plt.title(discrete[i])
    plt.suptitle('Data Distribution in Discrete Variables', fontsize = 20)
    plt.tight_layout()

BsmtHalfBath, KitchenAbvGr, and PoolArea features have imbalance distribution between their unique values. 

#### Relation between Discrete Features and Target

In [ ]:
for i in range(0, len(df_train_discrete.columns)):
    plt.subplot(8, 6, i+1)
    sns.boxplot(x=df_train_discrete[discrete[i]], y = target, data = df_train)
    plt.title(discrete[i])
    x=plt.xticks(rotation=90)
    plt.suptitle('Relation between Discrete Variables and Target', fontsize = 20)
    plt.tight_layout()

Some features such as OverallQual, BsmtFullBath, and TotRmsAbvGrd have positive correlation to Sale Price. 

### Numerical Variables

In [ ]:
df_train_numerical = df_train[numerical]

#### Data Distribution for Numerical Features

In [ ]:
for i in range(0, len(df_train_numerical.columns)):
    plt.subplot(5, 5, i+1)
    sns.histplot(df_train_numerical[numerical[i]], kde = True)
    plt.title(numerical[i])
    plt.suptitle('Data Distribution in Numerical Variables', fontsize = 20)
    plt.tight_layout()

Almost all features are positively skewed, concentrated around value of 0. 

#### Relation between Numerical Features and Target

In [ ]:
for i in range(0, len(numerical)):
    plt.subplot(5, 5, i+1)
    sns.regplot(x=df_train_numerical[numerical[i]], y=target, data = df_train, scatter_kws={'s':10}, line_kws={'color':'red'})
    plt.xlabel(numerical[i])
    plt.ylabel(target)
    plt.title(f'{numerical[i]} vs {target}')
    plt.suptitle('Relation between Numerical Variables and Target', fontsize = 20)
    plt.tight_layout()

plt.show()

Some features like LotFrontage, TotalBsmtSF, and GrLivArea have highly positive correlation to Sale Price. 

### Target Distribution

In [ ]:
sns.histplot(x=df_train[target], kde=True)
plt.title('SalePrice')
plt.tight_layout()

Sale Price have tendency to positively skewed, with the center around 100000 - 200000.

## Feature Engineering

### Missing Value Handling

In [ ]:
#checking missing values in percentage

missing = df_train.isnull().sum()
missing = missing[missing > 0]
missing = round(missing/len(df_train)*100,2)
missing.sort_values(ascending = False, inplace=True)
missing

We will drop columns that have more than 20% missing value.

PoolQC, MiscFeature, Alley, Fence, MasVnrType, and FireplaceQu will be dropped because more than 20% data in those columns are missing. 

In [ ]:
df_train = df_train.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'MasVnrType', 'FireplaceQu'], axis = 1)
df_test = df_test.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'MasVnrType', 'FireplaceQu'], axis = 1)

In [ ]:
# adjust variables after feature drop

remove_features = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'MasVnrType', 'FireplaceQu']

categorical = [item for item in categorical if item not in remove_features]

Even though the initial checking of missing value shows no missing values in all features, we will check thoroughly whether the dataset is not have "hidden" missing value, such as unknown or invalid data, by checking them through value counts for each features.

In [ ]:
for col in df_train.columns:
    print(f"============= {col} =================")
    display(df_train[col].value_counts())
    print()

After checking all unique values, there is no more missing or invalid values. 

### Impute Missing Value

Numerical and discrete features missing value will be imputed by their median, while categorical features will be imputed by their mode. 

In [ ]:
impute_numerical = SimpleImputer(strategy = 'median')
df_train[numerical] = impute_numerical.fit_transform(df_train[numerical])
df_test[numerical] = impute_numerical.transform(df_test[numerical])

impute_discrete = SimpleImputer(strategy = 'median')
df_train[discrete] = impute_discrete.fit_transform(df_train[discrete])
df_test[discrete] = impute_discrete.transform(df_test[discrete])

impute_categorical = SimpleImputer(strategy = 'most_frequent')
df_train[categorical] = impute_categorical.fit_transform(df_train[categorical])
df_test[categorical] = impute_categorical.transform(df_test[categorical])

### Check Duplicate Data

In [ ]:
df_train.duplicated().any()

No duplicate values found in the dataset. 

## Feature Selection

We will do feature selection for each data types.

#### Categorical 

Categorical Features will be selected by their mutual information scores. We will use features that have cumulative importance threshold of 80%. 

In [ ]:
def mutual_information_score(series):
    return mutual_info_score(series,df_train.SalePrice)

df_mi = df_train[categorical].apply(mutual_information_score)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')
mi_scores = pd.Series(df_mi['MI'], index=categorical)
mi_scores.sort_values(ascending=False, inplace=True)

df_mi.style.background_gradient(low=0.7, high=1.0,cmap='YlOrRd')

In [ ]:
# Sort scores and calculate cumulative importance
cumulative_importance = mi_scores.cumsum() / mi_scores.sum()

# Select features until cumulative importance threshold
cumulative_threshold = 0.8
selected_features = mi_scores[cumulative_importance <= cumulative_threshold].index

print("Selected features on categorical features based on cumulative importance:\n")
print(selected_features)

In [ ]:
df_train_categorical = df_train[selected_features]
df_train_categorical.columns

#### Discrete 

Discrete features will be selected based on variance. Features with low variance will be dropped, since it will not have significant relevance to modeling. Features will low variance will be checked through heatmap correlation, to analyze their correlation with target variable. 

In [ ]:
def highest_unique_value_and_percentage(column):
    value_counts = column.value_counts()
    highest_value = value_counts.idxmax()
    highest_count = value_counts.max()
    total_count = len(column)
    percentage = (highest_count / total_count) * 100
    return highest_value, percentage

# Apply the function to each categorical column
highest_values_and_percentages = {col: highest_unique_value_and_percentage(df_train_discrete[col]) for col in discrete}

for col, (value, percentage) in highest_values_and_percentages.items():
    print(f"Column: {col}, Highest Value: {value}, Percentage: {percentage:.2f}%")

Feature variance is represented by unique values percentage. BsmtHalfBath, PoolArea, KitchenAbvGr have dominant unique value (>90%) for 1 category, that related to low variance. 

Correlation Heatmap

In [ ]:
# heatmap correlation

corr = pd.concat([df_train_discrete, df_train['SalePrice']], axis=1).corr(method='spearman')

mask = np.triu(np.ones_like(corr, dtype=bool))

sns.heatmap(corr, cmap = 'Blues', annot=True, mask = mask)
plt.show()

After checking correlation between features and target, KitchenAbvGr has weak relationship, while BsmtHalfBath and PoolArea almost have no correlation at all. These two features will be dropped. Additionally, features MSSubClass, MoSold, YrSold also have almost no correlation with target, hence all these features will be dropped. The other features do not have multicolinearity to each other. 

In [ ]:
remove_discrete = ['MSSubClass', 'BsmtHalfBath', 'PoolArea', 'MoSold', 'YrSold']

df_train_discrete = df_train_discrete.drop(remove_discrete, axis = 1)

In [ ]:
discrete = [item for item in discrete if item not in remove_discrete]

df_train_discrete.columns

#### Numerical

Numerical Features will be selected based on variance inflation factor (VIF) and correlation.

In [ ]:
def calculate_vif(df):
    vif_data = pd.DataFrame()
    vif_data["Feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif_data

In [ ]:
df_train_numerical = df_train[numerical]

In [ ]:
# Calculate initial VIF

vif_data = calculate_vif(df_train_numerical)
print(f'{vif_data}\n')

features_removed = []

# Iteratively remove features with VIF > 10
# Feature that has highest VIF will be dropped one by one per iteration

while vif_data['VIF'].max() > 10:
    max_vif_feature = vif_data.loc[vif_data['VIF'].idxmax(), 'Feature']
    print(f"Removing feature with high VIF: {max_vif_feature}")
    features_removed.append(max_vif_feature)
    df_train_numerical = df_train_numerical.drop(columns=[max_vif_feature])
    vif_data = calculate_vif(df_train_numerical)

In [ ]:
print("Remaining features after VIF reduction:\n")
print(df_train_numerical.columns)
print(f'\nFinal VIF scores:\n{vif_data}')

Correlation Heatmap

In [ ]:
# heatmap correlation

corr = pd.concat([df_train_numerical, df_train['SalePrice']], axis=1).corr()

mask = np.triu(np.ones_like(corr, dtype=bool))

sns.heatmap(corr, cmap = 'Blues', annot=True, mask = mask)
plt.show()

There are no features that have high correlation to each other. GarageArea and MasVnrArea have positive moderate correlation with SalePrice.

## Outlier Handling

In [ ]:
# function to check histogram, distribution plot, and boxplot for each features

def check_plot(df, variable):
    # check distribution plot from variable in df.     

    # figure size and title
    plt.figure(figsize=(16, 4))
    plt.suptitle(f' Outlier Analysis for {variable} feature', fontsize=16, y=1.05)

    # histogram
    plt.subplot(1, 3, 1)
    sns.histplot(df[variable], bins = 30)
    plt.title('Histogram')

    # distribution (Q-Q) plot  
    plt.subplot(1, 3, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)
    plt.ylabel('Variable quantiles')

    # box plot
    plt.subplot(1, 3, 3)
    sns.boxplot(y=df[variable])
    plt.title('Boxplot')

    plt.show()


In [ ]:
# plot looping for distribution analysis each features

for col in df_train_numerical.columns:
    check_plot(df_train_numerical, col)

Almost all features have global and local outliers. IQR will be checked for each features for outlier handling. 

In [ ]:
# function to check upper IQR and lower IQR from columns

def find_outlier_boundary(df, variable):

    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)

    lower_boundary = df[variable].quantile(0.25) - (IQR * 1.5)
    upper_boundary = df[variable].quantile(0.75) + (IQR * 1.5)

    return upper_boundary, lower_boundary

In [ ]:
# dataframe to summarize IQR compared to minimum and maximum value for each columns

pd.DataFrame(data = {'Upper_IQR': [find_outlier_boundary(df_train_numerical, col)[0] for col in df_train_numerical.columns], 
                     'Maximum': [df_train_numerical[col].max() for col in df_train_numerical.columns], 
                     'Lower_IQR': [find_outlier_boundary(df_train_numerical, col)[1]  for col in df_train_numerical.columns], 
                     'Minimum': [df_train_numerical[col].min() for col in df_train_numerical.columns]}, 
             index = [col for col in df_train_numerical.columns])

Each values from features that is not in the range of Upper and Lower IQR will be replaced by their Upper and Lower IQR. 

In [ ]:
# replace the outliers with upper and lower IQR

for col in df_train_numerical.columns:
    Population_upper_limit, Population_lower_limit = find_outlier_boundary(df_train_numerical, col)
    
    df_train[col]= np.where(df_train[col] > Population_upper_limit, Population_upper_limit,
                       np.where(df_train[col] < Population_lower_limit, Population_lower_limit, df_train[col]))

In [ ]:
# plot looping for distribution analysis after outlier treatment in features

for col in df_train_numerical.columns:
    check_plot(df_train, col)

In [ ]:
# features that have singular value after outlier handling will be dropped. 

df_train_numerical = df_train_numerical.drop([ 'BsmtFinSF2', 'LowQualFinSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'MiscVal'], axis = 1)
df_train_numerical.columns

Final features that will be used for modeling

In [ ]:
final_features = list(df_train_categorical.columns) + list(df_train_numerical.columns) + list(df_train_discrete.columns)
final_features

## Data Train and Test Assignment

In [ ]:
# applying final features for model training

# X_train and y_train
X_train = df_train[final_features]
y_train = df_train['SalePrice']

#X_test and y_test
X_test = df_test[final_features]
y_test = df_test['SalePrice']

In [ ]:
X_train.head()

## Pipeline for Feature Encoding and Feature Scaling

In [ ]:
# Create Pipeline
# target encoding will be used instead of One Hot Encoding technique to prevent curse of dimensionality

preprocessor = Pipeline([
    ('target', TargetEncoder(cols= list(df_train_categorical.columns))),
    ('scaler', StandardScaler())
])

In [ ]:
X_train_processing = preprocessor.fit_transform(X_train, y_train)

X_test_processing = preprocessor.transform(X_test)

In [ ]:
# all features used in model training
 
preprocessor.get_feature_names_out()

In [ ]:
X_train_processing = pd.DataFrame(X_train_processing,columns=preprocessor.get_feature_names_out())
X_test_processing = pd.DataFrame(X_test_processing,columns=preprocessor.get_feature_names_out())

## Training Data

In [ ]:
# baseline and ensemble model for training

linear = LinearRegression() #baseline model
ada = AdaBoostRegressor(random_state=42)
rf = RandomForestRegressor(random_state=42)
xgb = XGBRegressor(random_state=42)

### Model Selection

In [ ]:
# model iteration
# two metrics will be used: 
# 1. RMSE 
# 2. R2 (for result interpretation clarity only)

ml = [('Linear Regression',linear),
      ('AdaBoost',ada),
      ('Random Forest',rf),
      ('XGBoost',xgb)]

r2_train = []
r2_test = []

rmse_train = []
rmse_test = []

models = []

for model in ml:
  model[1].fit(X_train_processing,y_train)

  predict_train = model[1].predict(X_train_processing)
  predict_test = model[1].predict(X_test_processing)

  r2_score_train = r2_score(y_train,predict_train)
  r2_score_test = r2_score(y_test,predict_test)

  rmse_score_train = root_mean_squared_error(y_train,predict_train)
  rmse_score_test = root_mean_squared_error(y_test,predict_test)

  r2_train.append(r2_score_train)
  r2_test.append(r2_score_test)

  rmse_train.append(rmse_score_train)
  rmse_test.append(rmse_score_test)

  models.append(model[0])

df_metrics = pd.DataFrame({'model':models,
                           'r2_score_train':r2_train,
                           'r2_score_test':r2_test,
                           'RMSE_train':rmse_train,
                           'RMSE_test':rmse_test})

In [ ]:
df_metrics

1. Linear Regression have similar value between high r2_score_train (0.834) and r2_score_test (0.832), which indicated no overfit and underfit. 

2. AdaBoost have similar value between high r2_score_train (0.853) and r2_score_test (0.849), which indicated no overfit and underfit. However, AdaBoost have higher R2 score and lower RMSE than Linear Regression. 

3. Random Forest have high difference between r2_score_train (0.977) and r2_score_test (0.873), which indicated this model is overfit. This model is categorized as overfit, indicating the model cannot understand general data pattern. 

4. XGBoost have extremely high difference between r2_score_train (0.999) and r2_score_test (0.891), which indicated this model is overfit. This model is categorized as overfit, indicating the model cannot understand general data pattern. 

As summary, Linear Regression and AdaBoost have stable performance between evaluation metrics in data train and data test, because it has low RMSE values, while also not showing any overfit or underfit tendency. Linear Regression is also has potential to be used, but the model performance is still behind AdaBoost. Even though Random Forest and XGBoost have overfit, they shows potential in outperforming both Linear Regression and AdaBoost by having higher r2 and lower RMSE. To decide which model is the best, one model from the best stable performance (AdaBoost) and one model from the best overfit model will be tuned to determine which one is better.

We will do hyperparameter tuning to get the best parameter of AdaBoost and XGBoost. 


# Hyperparameter Tuning Parameter AdaBoost

We will do separate tuning for each hyperparameter first, then integrate the best range to Optuna later. 

We will optimize log RMSE instead of RMSE, so the error in the small Sale Price will have more impact for model training. By doing this, the model will have better generalization in small Sale Price range. 

We will choose hyperparameter that give the lowest log RMSE, since the objective is to reduce the error as small as possible. 

### Learning Rate Tuning

In [ ]:
learning_rate_list = [0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

learning_rate = []
rmse_score = []


for eta in learning_rate_list:
  ada = AdaBoostRegressor(learning_rate=eta, random_state=42).fit(X_train_processing,y_train)
  rmse = root_mean_squared_error(np.log(y_test), np.log(ada.predict(X_test_processing)))
  rmse_score.append(rmse)
  learning_rate.append(eta)


df_learning_rate = pd.DataFrame({'learning_rate':learning_rate,
                       'rmse_score':rmse_score})

plt.figure(figsize=(8,6))
sns.pointplot(data=df_learning_rate,x='learning_rate',y='rmse_score')
plt.title('Tuning Learning Rate AdaBoost')
plt.show()

Optimal learning rate = 0.8

### n_estimators Tuning

In [ ]:
n_estimators = []
rmse_score = []

for estimator in [10, 25, 50, 75, 100, 250]:
  ada = AdaBoostRegressor(n_estimators=estimator, random_state=42).fit(X_train_processing,y_train)
  rmse = root_mean_squared_error(np.log(y_test), np.log(ada.predict(X_test_processing)))
  rmse_score.append(rmse)
  n_estimators.append(estimator)


df_estimator = pd.DataFrame({'n_estimators':n_estimators,
                       'rmse_score':rmse_score})

plt.figure(figsize=(8,6))
sns.pointplot(data=df_estimator,x='n_estimators',y='rmse_score')
plt.title('Tuning n_estimators AdaBoost')
plt.show()

Optimal n_estimators = 25

### Loss Tuning

In [ ]:
loss = []
rmse_score = []

for cat in ['linear', 'square', 'exponential']:
  ada = AdaBoostRegressor(loss=cat, random_state = 42).fit(X_train_processing,y_train)
  rmse = root_mean_squared_error(np.log(y_test), np.log(ada.predict(X_test_processing)))
  rmse_score.append(rmse)
  loss.append(cat)


df_loss = pd.DataFrame({'loss':loss,
                       'rmse_score':rmse_score})

plt.figure(figsize=(8,6))
sns.pointplot(data=df_loss,x='loss',y='rmse_score')
plt.title('Tuning loss AdaBoost')
plt.show()

Optimal loss  = exponential

### Model Performance Comparison with and without Tuning (AdaBoost)

#### Without Tuning

In [ ]:
ada = AdaBoostRegressor(random_state=42)
ada.fit(X_train_processing,y_train)


predict_train_without_tuning = ada.predict(X_train_processing)
predict_test_without_tuning = ada.predict(X_test_processing)

r2_score_train = r2_score(np.log(y_train), np.log(predict_train_without_tuning))
r2_score_test = r2_score(np.log(y_test), np.log(predict_test_without_tuning))

rmse_score_train = root_mean_squared_error(np.log(y_train), np.log(predict_train_without_tuning))
rmse_score_test = root_mean_squared_error(np.log(y_test), np.log(predict_test_without_tuning))

print('r2 score train',r2_score_train)
print('r2 score test',r2_score_test)

print('RMSE train',rmse_score_train)
print('RMSE test',rmse_score_test)

In [ ]:
r2_score_train_without_tuning = r2_score_train
r2_score_test_without_tuning = r2_score_test
rmse_score_train_without_tuning = rmse_score_train
rmse_score_test_without_tuning = rmse_score_test

#### With Tuning

In [ ]:
ada_tuning = AdaBoostRegressor(learning_rate = 0.8,
                  n_estimators = 25,
                  loss = 'exponential',
                  random_state=42)

ada_tuning.fit(X_train_processing,y_train)

predict_train_tuning = ada_tuning.predict(X_train_processing)
predict_test_tuning = ada_tuning.predict(X_test_processing)

r2_score_train = r2_score(np.log(y_train), np.log(predict_train_tuning))
r2_score_test = r2_score(np.log(y_test), np.log(predict_test_tuning))

rmse_score_train = root_mean_squared_error(np.log(y_train), np.log(predict_train_tuning))
rmse_score_test = root_mean_squared_error(np.log(y_test), np.log(predict_test_tuning))

print('r2 score train',r2_score_train)
print('r2 score test',r2_score_test)

print('RMSE train',rmse_score_train)
print('RMSE test',rmse_score_test)

Best parameters: learning_rate = 0.8,
                  n_estimators = 25,
                  loss = 'exponential',
                  random_state=42.

In [ ]:
r2_score_train_tuning = r2_score_train
r2_score_test_tuning = r2_score_test
rmse_score_train_tuning = rmse_score_train
rmse_score_test_tuning = rmse_score_test

We will compare the results from AdaBoost optimization with XGBoost later. 

# Hyperparameter Tuning Parameter XGBoost

### Learning Rate Tuning

In [ ]:
learning_rate_list = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

learning_rate = []
rmse_score = []


for eta in learning_rate_list:
  xgb = XGBRegressor(learning_rate=eta,random_state=42).fit(X_train_processing, y_train)
  rmse = root_mean_squared_error(np.log(y_test), np.log(xgb.predict(X_test_processing)))

  rmse_score.append(rmse)
  learning_rate.append(eta)


df_eta = pd.DataFrame({'learning_rate':learning_rate,
                       'rmse_score':rmse_score})

plt.figure(figsize=(8,6))
sns.pointplot(data=df_eta,x='learning_rate',y='rmse_score')
plt.title('Tuning Learning Rate XGBoost')
plt.show()

Optimal learning rate: 0.1

### Max_depth Tuning

In [ ]:
max_depth = []
rmse_score = []

for depth in range(1,13):
  xgb = XGBRegressor(max_depth=depth, random_state=42).fit(X_train_processing, y_train)
  rmse = root_mean_squared_error(np.log(y_test), np.log(xgb.predict(X_test_processing)))
  rmse_score.append(rmse)
  max_depth.append(depth)


df_max_depth = pd.DataFrame({'max_depth':max_depth,
                       'rmse_score':rmse_score})

plt.figure(figsize=(8,6))
sns.pointplot(data=df_max_depth,x='max_depth',y='rmse_score')
plt.title('Tuning max_depth XGBoost')
plt.show()

Optimal max_depth: 2

### N_estimator Tuning

In [ ]:
n_estimators = []
rmse_score = []

for estimator in [50, 75, 100, 125, 150, 175, 200]:
  xgb = XGBRegressor(n_estimators=estimator, random_state = 42).fit(X_train_processing, y_train)
  rmse = root_mean_squared_error(np.log(y_test), np.log(xgb.predict(X_test_processing)))
  rmse_score.append(rmse)
  n_estimators.append(estimator)


df_estimator = pd.DataFrame({'n_estimators':n_estimators,
                       'rmse_score':rmse_score})

plt.figure(figsize=(8,6))
sns.pointplot(data=df_estimator,x='n_estimators',y='rmse_score')
plt.title('Tuning n_estimators XGBoost')
plt.show()

Optimal n_estimators = 125

### Subsample Tuning

In [ ]:
subsample = []
rmse_score = []

for sub in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
  xgb = XGBRegressor(subsample=sub,random_state=42).fit(X_train_processing, y_train)
  rmse = root_mean_squared_error(np.log(y_test), np.log(xgb.predict(X_test_processing)))
  rmse_score.append(rmse)
  subsample.append(sub)


df_subsample = pd.DataFrame({'subsample':subsample,
                       'rmse_score':rmse_score})

plt.figure(figsize=(8,6))
sns.pointplot(data=df_subsample,x='subsample',y='rmse_score')
plt.title('Tuning subsample XGBoost')
plt.show()

Optimal subsample: 0.9

### Lambda Tuning

In [ ]:
reg_lambda = []
rmse_score = []

for lambda_ in [0.0001,0.001, 0.01, 0.1, 0.5, 1, 1.5, 1.6, 1.7, 1.8, 1.9, 2, 2.2, 2.5, 3]:
    xgb = XGBRegressor(reg_lambda=lambda_, random_state=42).fit(X_train_processing, y_train)
    rmse = root_mean_squared_error(np.log(y_test), np.log(xgb.predict(X_test_processing)))
    rmse_score.append(rmse)
    reg_lambda.append(lambda_)


#create dataset eta
df_reg_lambda = pd.DataFrame({'reg_lambda':reg_lambda,
                       'rmse_score':rmse_score})

plt.figure(figsize=(8,6))
sns.pointplot(data=df_reg_lambda,x='reg_lambda',y='rmse_score')
plt.title('Tuning reg_lambda XGBoost')
plt.show()

Optimal lambda = 1.6

### Model Performance Comparison with and without Tuning (XGBoost)

#### Without Tuning

In [ ]:
xgb = XGBRegressor(random_state=42)
xgb.fit(X_train_processing,y_train)


predict_train_without_tuning = xgb.predict(X_train_processing)
predict_test_without_tuning = xgb.predict(X_test_processing)

r2_score_train = r2_score(np.log(y_train), np.log(predict_train_without_tuning))
r2_score_test = r2_score(np.log(y_test), np.log(predict_test_without_tuning))

rmse_score_train = root_mean_squared_error(np.log(y_train), np.log(predict_train_without_tuning))
rmse_score_test = root_mean_squared_error(np.log(y_test), np.log(predict_test_without_tuning))

print('r2 score train',r2_score_train)
print('r2 score test',r2_score_test)

print('RMSE train',rmse_score_train)
print('RMSE test',rmse_score_test)

In [ ]:
r2_score_train_without_tuning_xgb = r2_score_train
r2_score_test_without_tuning_xgb = r2_score_test
rmse_score_train_without_tuning_xgb = rmse_score_train
rmse_score_test_without_tuning_xgb = rmse_score_test

#### With Tuning

In [ ]:
xgb_tuning = XGBRegressor(learning_rate = 0.1,
                  max_depth = 2,
                  n_estimators = 125,
                  subsample = 0.9,
                  reg_lambda = 1.6,
                  random_state=42)

xgb_tuning.fit(X_train_processing, y_train)

predict_train_tuning = xgb_tuning.predict(X_train_processing)
predict_test_tuning = xgb_tuning.predict(X_test_processing)

r2_score_train = r2_score(np.log(y_train), np.log(predict_train_tuning))
r2_score_test = r2_score(np.log(y_test), np.log(predict_test_tuning))

rmse_score_train = root_mean_squared_error(np.log(y_train), np.log(predict_train_tuning))
rmse_score_test = root_mean_squared_error(np.log(y_test), np.log(predict_test_tuning))

print('r2 score train',r2_score_train)
print('r2 score test',r2_score_test)

print('RMSE train',rmse_score_train)
print('RMSE test',rmse_score_test)

Best parameters: learning_rate = 0.1,
                  max_depth = 2,
                  n_estimators = 125,
                  subsample = 0.9,
                  reg_lambda = 1.6,
                  random_state=42.

In [ ]:
r2_score_train_tuning_xgb = r2_score_train
r2_score_test_tuning_xgb = r2_score_test
rmse_score_train_tuning_xgb = rmse_score_train
rmse_score_test_tuning_xgb = rmse_score_test

### Comparison Between AdaBoost and XGBoost Performance Before and After Hyperparameter Tuning

In [ ]:
index = ['AdaBoost R2 without Tuning', 'AdaBoost R2 with Tuning', 'XGBoost R2 without Tuning', 'XGBoost R2 with Tuning', 
         'AdaBoost RMSE without Tuning', 'AdaBoost RMSE with Tuning', 'XGBoost RMSE test without Tuning', 'XGBoost RMSE test with Tuning']
df_compare = pd.DataFrame({
    'Train': [r2_score_train_without_tuning, r2_score_train_tuning, r2_score_train_without_tuning_xgb, r2_score_train_tuning_xgb, 
              rmse_score_train_without_tuning, rmse_score_train_tuning, rmse_score_train_without_tuning_xgb, rmse_score_train_tuning_xgb],
    'Test': [r2_score_test_without_tuning, r2_score_test_tuning, r2_score_test_without_tuning_xgb, r2_score_test_tuning_xgb, 
              rmse_score_test_without_tuning, rmse_score_test_tuning, rmse_score_test_without_tuning_xgb, rmse_score_test_tuning_xgb]
}, index=index)

df_compare['Difference'] = abs(df_compare['Test'] - df_compare['Train'])

print('\nR2 and RMSE comparison without tuning and with tuning:')
df_compare

While AdaBoost provides robust and stable performance before and after tuning, XGBoost give significant improvement. At first XGBoost have extremely overfit before tuning. After tuning, the RMSE difference XGBoost is reduced to 0.04 from 0.15. 

After tuning, XGBoost has higher R2 score and lower RMSE than AdaBoost, while also improved from overfit to be an optimal model. Hence, XGBoost is the best model among the other models, and XGBoost will be further improved through hyperparameter tuning with Optuna. 

## Hyperparameter Tuning XGBoost using Optuna

The range of best parameters from before will be used in Optuna. 

In [ ]:
def objective(trial):
    param = {
        'objective': 'reg:squarederror',
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.3),
        'max_depth': trial.suggest_int('max_depth', 2, 3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 175),
        'subsample': trial.suggest_float('subsample', 0.8, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 1.5, 1.7),
    }

    model = XGBRegressor(**param,random_state=42)
    model.fit(X_train_processing, y_train, eval_set=[(X_test_processing, y_test)], early_stopping_rounds=50, verbose=False)

    # RMSE calculation
    preds = model.predict(X_test_processing)
    rmse = root_mean_squared_error(np.log(y_test), np.log(preds))
    
    return rmse

study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))  
study.optimize(objective, n_trials=100)

print('Best trial:', study.best_trial.params)

In [ ]:
best_params = study.best_trial.params
best_params

In [ ]:
# hyperparameter importance from Optuna

vis.plot_param_importances(study)

The importance of max_depth learning is the highest, which indicated this as the high importance feature that needs to be tuned. 

In [ ]:
xgb_optuna = XGBRegressor(**best_params,
                  random_state=42)

xgb_optuna.fit(X_train_processing,y_train)

predict_train_optuna = xgb_optuna.predict(X_train_processing)
predict_test_optuna = xgb_optuna.predict(X_test_processing)

r2_score_train = r2_score(np.log(y_train), np.log(predict_train_optuna))
r2_score_test = r2_score(np.log(y_test), np.log(predict_test_optuna))

rmse_score_train = root_mean_squared_error(np.log(y_train), np.log(predict_train_optuna))
rmse_score_test = root_mean_squared_error(np.log(y_test), np.log(predict_test_optuna))

print('r2 score train', r2_score_train)
print('r2 score test', r2_score_test)

print('RMSE train', rmse_score_train)
print('RMSE test', rmse_score_test)

In [ ]:
r2_score_train_optuna = r2_score_train
r2_score_test_optuna = r2_score_test
rmse_score_train_optuna = rmse_score_train
rmse_score_test_optuna = rmse_score_test

## Final Comparison of Hyperparameter Tuning

In [ ]:
index = ['R2 without Tuning', 'R2 with Tuning', 'R2 Optima', 'RMSE without Tuning', 'RMSE Tuning', 'RMSE Optuna']
df_compare = pd.DataFrame({
    'Train': [r2_score_train_without_tuning_xgb, r2_score_train_tuning_xgb, r2_score_train_optuna, rmse_score_train_without_tuning_xgb, rmse_score_train_tuning_xgb, rmse_score_train_optuna],
    'Test': [r2_score_test_without_tuning_xgb, r2_score_test_tuning_xgb, r2_score_test_optuna, rmse_score_test_without_tuning_xgb, rmse_score_test_tuning_xgb, rmse_score_test_optuna]
}, index=index)

df_compare['Difference'] = abs(df_compare['Test'] - df_compare['Train'])

print('\nR2 and RMSE comparison without tuning, with tuning, and tuning with Optuna:')
df_compare

### Visualization for Test Dataset Prediction using Optuna

In [ ]:
# actual vs predicted house sale prices in test dataset
final_result = pd.DataFrame({'actual': y_test, 'predicted': predict_test_optuna})
final_result.head()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x=final_result['actual'], y=final_result['predicted'])
plt.plot([final_result['actual'].min(), final_result['actual'].max()], [final_result['actual'].min(), final_result['actual'].max()], 'r--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted House Sale Prices')
plt.show()

As we can see, almost all predictions is near the straight line, which is a good sign the model predicted well. 

## Summary

Initially, XGBoost model performance is overfit, then improved significantly after hyperparameter tuning. Both RMSE train and test are decreased after tuning. The difference between R2 score and RMSE train and test also become smaller after hyperparameter tuning, with the best performance from hyperparameter tuning with Optuna. 

After comparing R2 and RMSE values, we will use model with Optuna tuning as the final best model, since it give the lowest RMSE value, the highest R2, while also give the smallest difference between train and test. The best parameters that will be used for house price prediction are based on Optuna tuning: 

In [ ]:
print('Best Hyperparameters:')
best_params

## Use Data Test for House Prediction

In [ ]:
# read data test

data_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
ids = data_test.pop('Id')

In [ ]:
# reassign variables for data test prediction

categorical_test = [col for col in df_train.columns if df[col].dtype not in ['float64','int64']]
discrete_test = [col for col in df_train.columns if df[col].nunique() < 20 and col not in categorical_test]
numerical_test = [col for col in df_train.columns if col not in categorical_test + discrete_test + ['SalePrice']]

In [ ]:
# impute data test based on train dataset information

data_test[numerical_test] = impute_numerical.transform(data_test[numerical_test])
data_test[discrete_test] = impute_discrete.transform(data_test[discrete_test])
data_test[categorical_test] = impute_categorical.transform(data_test[categorical_test])

In [ ]:
# use the same column as train

data_test = data_test[final_features]

In [ ]:
# preprocessing data test based on train dataset information

data_test_prediction = preprocessor.transform(data_test)
data_test_prediction = pd.DataFrame(data_test_prediction,columns=preprocessor.get_feature_names_out())

In [ ]:
# predict data test using the best model 

pred = xgb_optuna.predict(data_test_prediction) 

In [ ]:
# input prediction result to dataframe for submission

df_submit = pd.DataFrame(pred, index=ids, columns=['SalePrice']).reset_index()
df_submit.head()

In [ ]:
# save prediction to sample_submission.csv

df_submission = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
df_submission['SalePrice'] = df_submit['SalePrice']
df_submission.to_csv('/kaggle/working/submission.csv', index=False)

Thank you for exploring my notebook! I hope you found it informative and that it enriched your knowledge. If you have any questions, suggestions, or feedback, please do not hesitate to reach out. I genuinely value your input! 📘✨

If you found this notebook helpful, I would greatly appreciate an upvote. Your support encourages me to continue creating more high-quality content! 👍😊